# Visual 1: SPY Momentum vs VIX (High Volatility Regime Shading)

In [5]:
# import libraries
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [15]:
# Load the master_df
master_df = pd.read_csv("master_df.csv")

# Rename the column to 'Date' and set it as index
master_df = master_df.rename(columns={'Unnamed: 0': 'Date'})
master_df['Date'] = pd.to_datetime(master_df['Date'])

# Preview the first 5 rows
master_df.head()

,Date,"('Close', 'IWM')","('Close', 'QQQ')","('Close', 'SPY')","('High', 'IWM')","('High', 'QQQ')","('High', 'SPY')","('Low', 'IWM')","('Low', 'QQQ')","('Low', 'SPY')","('Open', 'IWM')","('Open', 'QQQ')","('Open', 'SPY')","('Volume', 'IWM')","('Volume', 'QQQ')","('Volume', 'SPY')",VIX
0,2005-01-03,48.436901,33.738796,81.847145,49.422161,34.413573,82.840468,48.179218,33.627756,81.575001,49.335005,34.242743,82.704393,16062600.0,100970900.0,55748000.0,14.08
1,2005-01-04,47.398605,33.123806,80.846985,48.569548,33.943789,82.010397,47.315237,32.927353,80.581646,48.531657,33.883996,81.955967,27450000.0,136623200.0,69167600.0,13.98
2,2005-01-05,46.451225,32.918808,80.289101,47.474379,33.277548,81.132744,46.451225,32.859018,80.282296,47.345539,33.038388,80.785759,29884200.0,127925500.0,65667300.0,14.09
3,2005-01-06,46.697544,32.756527,80.697296,47.140910,33.064020,81.064690,46.401964,32.747987,80.459172,46.739226,32.995690,80.581636,23061200.0,102934600.0,47814700.0,13.58
4,2005-01-07,46.178394,32.927345,80.581650,46.951445,33.200672,81.119134,46.151868,32.636935,80.370735,46.803656,32.935889,80.942239,20906200.0,123104000.0,55847700.0,13.49


In [20]:
# --- Compute daily SPY Momentum (% change in SPY Close) ---
master_df["SPY_Momentum"] = master_df["('Close', 'SPY')"].pct_change() * 100

# --- Preview to confirm column creation ---
print("SPY Momentum calculated. Preview:")
display(master_df[["Date", "('Close', 'SPY')", "SPY_Momentum"]].head(10))

SPY Momentum calculated. Preview:


,Date,"('Close', 'SPY')",SPY_Momentum
0,2005-01-03,81.847145,NaN
1,2005-01-04,80.846985,-1.221985
2,2005-01-05,80.289101,-0.690050
3,2005-01-06,80.697296,0.508407
4,2005-01-07,80.581650,-0.143309
5,2005-01-10,80.962692,0.472865
6,2005-01-11,80.404739,-0.689148
7,2005-01-12,80.670090,0.330018
8,2005-01-13,80.023781,-0.801175
9,2005-01-14,80.445587,0.527101


In [21]:
# --- Define the 80th percentile VIX threshold (high-volatility marker) ---
vix_threshold = master_df["VIX"].quantile(0.80)

print(f"VIX high-volatility threshold (80th percentile): {vix_threshold:.2f}")

# Show basic VIX stats for context
print("\nVIX Summary Statistics:")
display(master_df["VIX"].describe().to_frame().T)

VIX high-volatility threshold (80th percentile): 23.75

VIX Summary Statistics:


,count,mean,std,min,25%,50%,75%,max
VIX,5219.0,19.161795,8.695743,9.14,13.42,16.7,22.17,82.69


In [22]:
# --- Create a subplot with dual Y-axes for SPY Momentum and VIX ---
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add SPY Momentum trace
fig.add_trace(
    go.Scatter(
        x=master_df["Date"],
        y=master_df["SPY_Momentum"],  # ✅ This column now exists
        mode="lines",
        name="SPY Momentum (%)",
        line=dict(color="blue"),
        hovertemplate="Date: %{x|%Y-%m-%d}<br>Momentum: %{y:.2f}%<extra></extra>"
    ),
    secondary_y=False,
)

# Add VIX trace
fig.add_trace(
    go.Scatter(
        x=master_df["Date"],
        y=master_df["VIX"],
        mode="lines",
        name="VIX",
        line=dict(color="red"),
        hovertemplate="Date: %{x|%Y-%m-%d}<br>VIX: %{y:.2f}<extra></extra>"
    ),
    secondary_y=True,
)

print("Plot initialized with SPY Momentum and VIX traces.")

Plot initialized with SPY Momentum and VIX traces.


In [23]:
# --- Identify high-volatility periods and shade them ---
high_vol = master_df["VIX"] >= vix_threshold
shaded_periods = []

for i in range(1, len(master_df) - 1):
    if high_vol.iloc[i] and not high_vol.iloc[i - 1]:
        start = master_df["Date"].iloc[i]
    if high_vol.iloc[i] and not high_vol.iloc[i + 1]:
        end = master_df["Date"].iloc[i]
        shaded_periods.append((start, end))
        fig.add_vrect(
            x0=start, x1=end,
            fillcolor="rgba(255, 99, 71, 0.15)",
            line_width=0,
            annotation_text="High Vol",
            annotation_position="top left",
            annotation_font_size=10
        )

print(f"Number of high-volatility regimes shaded: {len(shaded_periods)}")
if shaded_periods:
    print("Example periods:")
    for s, e in shaded_periods[:3]:
        print(f"  {s.date()} → {e.date()}")

Number of high-volatility regimes shaded: 101
Example periods:
  2006-06-13 → 2006-06-13
  2007-07-27 → 2007-07-27
  2007-08-03 → 2007-08-03


In [24]:
# --- Finalize layout and display interactive chart ---
fig.update_layout(
    title="SPY Momentum vs VIX (High Volatility Regime Shading)",
    xaxis_title="Date",
    yaxis_title="SPY Momentum (%)",
    yaxis2_title="VIX",
    hovermode="x unified",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    template="plotly_white"
)

fig.show()

# --- Optional: export plot to HTML ---
fig.write_html("SPY_Momentum_vs_VIX.html")
print("Interactive chart saved as 'SPY_Momentum_vs_VIX.html'")

Interactive chart saved as 'SPY_Momentum_vs_VIX.html'
